# ДЗ1 Зиновьева Ольга

Я решила парсить отзывы на книги на [Литресе](https://www.litres.ru). Для честности брала именно книги, а не аудиокниги (исключая вероятность, что в отзывах на аудиокниги внимание может уделяться голосу диктора)

Этот скрипт будет запускаться только:
- не в колабе! он не дружит с селениумом
- если у вас есть firefox на компе

Эти условия жесткие, поэтому в репо я положила json-ы с крауленными отзывами. Думаю, их названия довольно понятные) Дальше по коду найдите блок с чтением из json-ов и начинайте с него) А если сами запускаете краулинг, то игнорируйте этот блок

Почему я юзаю pymorphy3, а не pymorphy2? Потому что у меня на компе стоит 11 питон, а он с pymorphy2 не дружит. [Вот на stackoverflow про это](https://ru.stackoverflow.com/questions/1479188/Почему-не-работает-пакет-pymorphy2-на-python-3-11)

In [ ]:
!pip install pymorphy3

In [ ]:
!pip install nltk

In [ ]:
!pip install selenium

In [132]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import random
import nltk
from nltk.tokenize import RegexpTokenizer
import pymorphy3
import json
from typing import List, Set, Dict, Optional

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [229]:
def get_soup(url: str) -> BeautifulSoup:
    """ Универсальная функция для краулинга странички по ссылке
    """
    dr =  webdriver.Firefox()
    dr.get(url)
    soup = BeautifulSoup(dr.page_source)
    time.sleep(2)
    dr.quit()
    time.sleep(3)
    return soup

План такой:

Я стартую с жанровой странички (потому что на главной не нашла ссылок на жанровые странички).
Дальше иду по жанрам и краулю книжки с их первой странички. И для каждой книги краулю первую страницу с отзывами.

Почему везде только первые страницы? Потому что на литресе промотка страницы сопровождается изменением query, а не пути. А селениум, как и requests, с query не дружат)

In [230]:
soup = get_soup('https://www.litres.ru/genre/legkoe-chtenie-201583/')

In [231]:
main_url = 'https://www.litres.ru'

Для сбора отзывов использую множества, чтобы исключить возможность повторения (в случае, если книга относится к 2 жанрам)

In [232]:
positive = set()
negative = set()

In [233]:
for genre in tqdm(soup.find('div', {'class': 'AllGenres-module__genres-wrapper'}).find_all('a')[1:16]):
    genre_soup = get_soup(main_url + genre['href'])

    href_blocks = genre_soup.find_all('a', {'class': "bookname descr-no_js"})
    # Почему-то на разных страницах разные классы у блоков с ссылками. (Кто это писал вообще?)
    if not href_blocks:
        href_blocks = genre_soup.find_all('a', {'class': "art__name__href"})

    # Список ссылок на страницы книг с 1 страницы жанра
    hrefs = [main_url + block['href'] + 'otzivi/' for block in href_blocks if 'audiobook' not in block['href']]

    for href in hrefs:
        book_soup = get_soup(href)
        for review in book_soup.find_all('article', {'class': 'Comment-module__comment'}):
            stars = review.find_all('div', {'class': 'Stars-modules__star Stars-modules__orangeTheme'})

            # Считаем цветные звездочки, если отзыв со звездочками. Решила, что 4+ - позитивный, 2- - негативный
            if len(stars):
                rate = 0
                for star in stars:
                    if star.find('path', {'class': "Stars-modules__svgPath Stars-modules__svgPathActive"}):
                        rate += 1
                text = review.find('div', {'class': 'Comment-module__reviewText'}).text
                if rate < 3:
                    negative.add(text)
                elif rate > 3:
                    positive.add(text)

100%|██████████| 15/15 [2:40:08<00:00, 640.58s/it] 


In [234]:
with open('pos.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(positive), ensure_ascii=False))

with open('neg.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(negative), ensure_ascii=False))

Вот отсюда надо начинать, если вы не запускаете краулинг. А если не запускаете, то не выполняйте этот блок!

In [235]:
with open('pos.json', 'r', encoding='utf-8') as f:
    positive = json.loads(f.read())

with open('neg.json', 'r', encoding='utf-8') as f:
    negative = json.loads(f.read())

In [237]:
print('Положительных -', len(positive), '; отрицательных -', len(negative))

Положительных - 1562 ; отрицательных - 173


Положительных прямо намного больше, тк мы брали везде первые странички, а там всегда все с более высоким рейтингом. Поэтому берем рандомный сэмпл положительных по длине отрицательных

In [238]:
pos = random.sample(sorted(positive), len(negative))
neg = negative

In [239]:
test_sample = {}
for samp in random.sample(sorted(pos), 15):
    test_sample[samp] = 'pos'
for samp in random.sample(sorted(neg), 15):
    test_sample[samp] = 'neg'

pos_sample = {samp for samp in pos if samp not in test_sample}
neg_sample = {samp for samp in neg if samp not in test_sample}

In [240]:
tokenizer = RegexpTokenizer(r'\w+')

In [241]:
def tokenize_several_texts(sample: Set[str]) -> List[List[str]]:
    """ Функция для токенизации сэмпла текстов
    """
    return [tokenizer.tokenize(text.lower()) for text in sample]

In [242]:
def make_frequency_dicts(tokenized_texts: List[List[str]]) -> Dict[str, int]:
    """ Функция для создания частотных словарей по массиву токенизованных текстов
    """
    freq_dict = {}
    for text in tokenized_texts:
        for token in text:
            freq_dict.setdefault(token, 0)
            freq_dict[token] += 1
    return freq_dict

In [243]:
def make_set_of_unique_tonality_tokens(
        this_tonality: Dict[str, int],
        different_tonality: Dict[str, int],
        freq_threshold: int,
        stop_words:Optional[Set[str]]=set()
) -> Set[str]:
    """ Функция для получения множества токенов тональности A, которые не присутствуют в отзывах тональности B
    """
    return set(
        [key for key, item in this_tonality.items()
         if key not in different_tonality
         and item > freq_threshold
         and key not in stop_words]
    )

In [244]:
pos_tokenized = tokenize_several_texts(pos_sample)
neg_tokenized = tokenize_several_texts(neg_sample)

In [245]:
pos_frequency = make_frequency_dicts(pos_tokenized)
neg_frequency = make_frequency_dicts(neg_tokenized)

In [246]:
pos_unique = make_set_of_unique_tonality_tokens(pos_frequency, neg_frequency, 2)
neg_unique = make_set_of_unique_tonality_tokens(neg_frequency, pos_frequency, 2)

Токенизируем также тестовые тексты

In [247]:
test_tokenized = tokenize_several_texts(test_sample.keys())
test_tonality = list(test_sample.values())

In [248]:
def calc_accuracy(test_texts: List[List[str]], test_labels: List[str], pos: Set[str], neg: Set[str]) -> float:
    """ Функция, которая считает accuracy модели определения тональности для тестового сэмпла на основании уникальных для pos и neg отзывов токенов"""
    true_guess = 0

    for i, text in enumerate(test_texts):
        score = 0
        for token in text:
            if token in pos:
                score += 1
            elif token in neg:
                score -= 1
        true_guess += (test_labels[i] == ('pos' if score > 0 else 'neg' if score < 0 else 'neu'))

    return true_guess / len(test_texts)

Это результат для tokenized + нижнерегистрного текста

In [249]:
calc_accuracy(test_tokenized, test_tonality, pos_unique, neg_unique)

0.7

Теперь про потенциальные способы улучшить accuracy:

- лемматизировать лексику в тональных словарях и тестовом сэмпле
- убрать из тональных словарей стоп-слова
- увеличить для тональных словарей порог частотности для входа

а еще можно брать в тональный словарь не только токены, которые не встретились в текстах другой тональности, но еще и слова, у которых стат. значимо больше вхождений в тексты рассматриваемой тональности. но это я делать не буду)

В общем, сперва пробую лемматизацию с pymorphy

In [250]:
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [251]:
def lemmatize_texts(tokenized_texts: List[List[str]]) -> List[List[str]]:
    """ Функция, которая лемматизирует токены в каждом из текстов
    """
    return [[morph.parse(token)[0].normal_form for token in text] for text in tokenized_texts]

In [252]:
pos_lemmatized = lemmatize_texts(pos_tokenized)
neg_lemmatized = lemmatize_texts(neg_tokenized)

In [253]:
pos_lemmatized_frequency = make_frequency_dicts(pos_lemmatized)
neg_lemmatized_frequency = make_frequency_dicts(neg_lemmatized)

In [254]:
pos_lemmatized_unique = make_set_of_unique_tonality_tokens(pos_lemmatized_frequency, neg_lemmatized_frequency, 2)
neg_lemmatized_unique = make_set_of_unique_tonality_tokens(neg_lemmatized_frequency, pos_lemmatized_frequency, 2)

In [255]:
test_lemmatized = lemmatize_texts(test_tokenized)

Итак, лемматизация особо не изменила ситуацию (запускала несколько раз, оно то лучше, то хуже (слава рандому))

In [257]:
calc_accuracy(test_lemmatized, test_tonality, pos_lemmatized_unique, neg_lemmatized_unique)

0.6666666666666666

Теперь попробуем убрать стоп-слова из словарей

In [258]:
stop_words = set(nltk.corpus.stopwords.words("russian"))

In [259]:
pos_no_stopwords_unique = make_set_of_unique_tonality_tokens(pos_frequency, neg_frequency, 2, stop_words)
neg_no_stopwords_unique = make_set_of_unique_tonality_tokens(neg_frequency, pos_frequency, 2, stop_words)

Тут то же, что и с лемматизацией. В зависимости от рандома то чуть лучше, то чуть хуже

In [260]:
calc_accuracy(test_tokenized, test_tonality, pos_no_stopwords_unique, neg_no_stopwords_unique)

0.7

Еще попробуем повысить порог частотности для токена, чтобы войти в словарь. Вот тут всегда точность ниже

In [261]:
pos_higher_freq_unique = make_set_of_unique_tonality_tokens(pos_frequency, neg_frequency, 3)
neg_higher_freq_unique = make_set_of_unique_tonality_tokens(neg_frequency, pos_frequency, 3)

In [262]:
calc_accuracy(test_tokenized, test_tonality, pos_higher_freq_unique, neg_higher_freq_unique)

0.5666666666666667